In [6]:
import pandas as pd
import os
import cv2
from PIL import Image

from detectron2.utils.logger import setup_logger
setup_logger()

from hackathon.predict import get_model_for_eval

from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

In [7]:
# get validation set images
home = "../data/rockfish_labels/"
labels = os.path.join(home, "jimmy_lynker_5k_25k.csv")
images = "../data/"

v020 = pd.read_csv(labels)

In [8]:
val = v020[(v020.is_validation == True)]
val

,filename,x_min,x_max,y_min,y_max,title,value,is_validation
10,rockfish/OCNMS_port/20170819.185225.00691.tif,571,869,546,832,NaN,NaN,True
11,rockfish/OCNMS_port/20170819.185225.00691.tif,1709,1857,1546,1864,NaN,NaN,True
47,rockfish/OCNMS_port/20170819.192202.01357.tif,1230,1641,1682,1871,NaN,NaN,True
51,rockfish/OCNMS_port/20170819.192650.01465.tif,1852,2088,609,868,NaN,NaN,True
52,rockfish/OCNMS_port/20170819.192650.01465.tif,211,338,900,1116,NaN,NaN,True
...,...,...,...,...,...,...,...,...
24394,rockfish/OCNMS_port/20170902.230139.04240.tif,41,261,1808,2028,NaN,NaN,True
24398,rockfish/OCNMS_port/20170827.173835.00629.tif,673,893,1566,1786,NaN,NaN,True
24427,rockfish/OCNMS_port/20170820.173050.00364.tif,412,632,1745,1965,NaN,NaN,True
24437,rockfish/OCNMS_port/20170901.181557.01071.tif,1938,2158,1077,1297,NaN,NaN,True


In [5]:
predictor = CPUPredictor(cfg)

NameError: name 'CPUPredictor' is not defined

In [9]:
# Get the configuration ready
model_dir = "../models/"
model_names = [
    "frcnn-r101-pct25", 
    "frcnn-r101-pct50", 
    "frcnn-r101-pct100"
]

predictors = []

cfg = get_model_for_eval("frcnn-r101-pct10", model_dir)
predictors.append({
    "frcnn-r101-pct10": DefaultPredictor(cfg)
})

for model_name in model_names:
    cfg = get_model_for_eval(model_name, model_dir)
    predictor = DefaultPredictor(cfg)
    predictors.append({
        model_name: predictor
    })


[12/09 12:05:25 detectron2]: Rank of current process: 0. World size: 1
[12/09 12:05:25 detectron2]: Rank of current process: 0. World size: 1
[12/09 12:05:27 detectron2]: Environment info:
----------------------  -------------------------------------------------------------------------------------
sys.platform            linux
Python                  3.7.8 | packaged by conda-forge | (default, Nov 27 2020, 19:24:58) [GCC 9.3.0]
numpy                   1.19.4
detectron2              0.3 @/home/ubuntu/anaconda3/envs/noaa/lib/python3.7/site-packages/detectron2
Compiler                GCC 7.3
CUDA compiler           CUDA 10.1
detectron2 arch flags   3.7, 5.0, 5.2, 6.0, 6.1, 7.0, 7.5
DETECTRON2_ENV_MODULE   <not set>
PyTorch                 1.6.0+cu101 @/home/ubuntu/anaconda3/envs/noaa/lib/python3.7/site-packages/torch
PyTorch debug build     False
GPU available           True
GPU 0,1,2,3,4,5,6,7     Tesla V100-SXM2-16GB (arch=7.0)
CUDA_HOME               /usr/local/cuda
Pillow             

RuntimeError: CUDA out of memory. Tried to allocate 50.00 MiB (GPU 0; 15.78 GiB total capacity; 648.75 MiB already allocated; 34.75 MiB free; 662.00 MiB reserved in total by PyTorch)

In [12]:
def show_model_output(predictors, df_idx=None):
    '''
    
    '''
    size = 2448, 2050
    val_sample = val.sample(1)
    if df_idx:
        val_sample = pd.DataFrame(val.loc[df_idx]).T
    display(val_sample)
    val_filename = os.path.join(images, val_sample.filename.values[0])
    val_image = cv2.imread(val_filename)
    output_list = []
    for i, predictor in enumerate(predictors):
        predictor_key = list(predictor.keys())[0]
        outputs = predictor[predictor_key](val_image)
        v = Visualizer(val_image[:,:,::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]))
        v = v.draw_instance_predictions(outputs['instances'].to('cpu'))
        img = v.get_image()
        img = Image.fromarray(img)
        img.thumbnail(size, Image.ANTIALIAS)
        output_list.append({
            predictor_key : outputs
        })
        display(predictor_key)
        display(val_filename)
        img.save("vis_sample_{}.tif".format(i), "TIFF")
        display(img)
        
    return output_list
    

In [13]:
def avg_output_confidence(output_list):
    '''Calculate mean confidence of predictor output'''
    for out in output_list:
        model_name = list(out.keys())[0]
        avg_score = list(out.values())[0]['instances'].scores.to('cpu').mean()
        print(model_name, " average confidence:", float(avg_score))

In [ ]:
# difficult example - starfish: 11737
# compelling... 13126
# pretty 18951

In [14]:
olist = show_model_output(predictors, 13126)
olist
avg_output_confidence(olist)

,filename,x_min,x_max,y_min,y_max,title,value,is_validation
13126,rockfish_full/SHE-11/Fish_Crawlers_NonAttached...,1899,2066,724,908,Animal,animal,True


RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 15.78 GiB total capacity; 666.24 MiB already allocated; 14.75 MiB free; 682.00 MiB reserved in total by PyTorch)